In [1]:
import numpy as np
from tqdm import tqdm
import random

In [2]:
dataset = np.load("../data/formatted/dataset.npy", allow_pickle=True)
midi_dataset = np.load("../data/formatted/midi_dataset.npy", allow_pickle=True)
meta_dataset = np.load("../data/formatted/meta_augmented.npy", allow_pickle=True)

print(dataset.shape, midi_dataset.shape, meta_dataset.shape)

(56412, 1024, 2) (56412, 1024, 3) (56412,)


In [4]:
print(dataset[0])

[['Style' 0.0]
 ['Medium Swing' 0.0]
 ['Tonality' 0.0]
 ...
 ['<pad>' 0.0]
 ['<pad>' 0.0]
 ['<pad>' 0.0]]


In [4]:
#Check both dataset are equivalent 
x, y = dataset.shape[0], dataset.shape[1]

for i in range (x):
    for j in range (y):
        element = dataset[i][j][0]
        check = midi_dataset[i][j][2] 
        if element != check:
            print("Error at ", i, j)
            print(element, check)
            break   
        
print("Dataset are equivalent")

In [28]:
#Token from dataset

chords_array = [[item[0] for item in row] for row in dataset]
concatenated_array = np.hstack(chords_array) 
tokens = set(concatenated_array)

print(len(tokens))
print(tokens)
np.save("../data/formatted/tokens.npy", list(tokens))

243
{'o', ':|', 'Moderately', 'Medium Up Swing', 'Traditional Gospel', 'Chacarera', 'Medium Rock', 'C', 'Jazz', 'Cuban Son', 'Folk', 'Blues Rock', '<Start>', 'add b9', 'Funk Jazz', 'B major', 'Db minor', "R'n'B", 'Medium Blues', 'E##', 'Slow Ballad', 'dom7', 'Db', 'Rock/Reggae', 'A minor', 'D', 'Afro', 'Form_C', 'Rock/Hip Hop', 'add 9', 'Medium Swing', 'aug', 'Rock Blues', 'Moderate Latin', 'Shuffle Blues', 'Bbb', 'Gypsy Swing', 'Up Swing', 'B##', 'UP Swing', 'Afoxé-Samba', 'D##', 'Soul', 'o_maj7', 'alter b5', 'Up Tempo Swing', 'Samba-Rock', 'Waltz', 'Afoxe', 'add 2', 'D#', 'Choro - Samba', 'Form_A', 'Up Samba', 'Rock Slow', 'Up Waltz (One Feel)', 'March', 'Slow Swing', 'maj6', 'Blues Pop', 'Salsa', 'add 11', 'Bossa Nova', 'add #5', 'D major', 'E#', 'add b13', 'Pop Shuffle', 'Electric Blues', 'Blues', 'm6', 'Bossa Acoustic', 'Gb minor', 'Electro Pop', 'Fb', 'sus4', 'm7', '<End>', 'Gypsy Bossa', 'Samba Funk', 'Rock-Folk', 'Medium Slow', 'Medium Waltz', 'alter #11', 'Form_D', 'Abb', 'Del

In [9]:
stoi = { tk:i for i,tk in enumerate(tokens) }
itos = { i:tk for i,tk in enumerate(tokens) }

print(stoi)

{'o': 0, ':|': 1, 'Moderately': 2, 'Medium Up Swing': 3, 'Traditional Gospel': 4, 'Chacarera': 5, 'Medium Rock': 6, 'C': 7, 'Jazz': 8, 'Cuban Son': 9, 'Folk': 10, 'Blues Rock': 11, '<Start>': 12, 'add b9': 13, 'Funk Jazz': 14, 'B major': 15, 'Db minor': 16, "R'n'B": 17, 'E##': 18, 'Medium Blues': 19, 'Slow Ballad': 20, 'dom7': 21, 'Db': 22, 'Rock/Reggae': 23, 'A minor': 24, 'D': 25, 'Afro': 26, 'Form_C': 27, 'Rock/Hip Hop': 28, 'add 9': 29, 'Medium Swing': 30, 'aug': 31, 'Rock Blues': 32, 'Moderate Latin': 33, 'Shuffle Blues': 34, 'Bbb': 35, 'Gypsy Swing': 36, 'B##': 37, 'Up Swing': 38, 'Fast Swing': 39, 'UP Swing': 40, 'Afoxé-Samba': 41, 'Gospel Ballad': 42, 'D##': 43, 'Soul': 44, 'o_maj7': 45, 'alter b5': 46, 'Up Tempo Swing': 47, 'Samba-Rock': 48, 'Waltz': 49, 'Afoxe': 50, 'add 2': 51, 'D#': 52, 'Choro - Samba': 53, 'Form_A': 54, 'Up Samba': 55, 'Rock Slow': 56, 'Up Waltz (One Feel)': 57, 'March': 58, 'Slow Swing': 59, 'maj6': 60, 'Blues Pop': 61, 'Rock Waltz': 62, 'add 11': 63, 'Bo

In [13]:
#Get the number of real songs
realSongs = len(dataset)/12
tenPercent = int(0.1 * realSongs)

#random a number without repeating number
randomList = random.sample(range(0, int(realSongs)), tenPercent)

#if number is bigger than 12 multiply it by 12
for i in range(len(randomList)):
    randomList[i] = randomList[i] * 12

#populate a random list with the 12 subsequent numbers per value
final_random_list=[]
for number in randomList:
    for i in range(12):
        final_random_list.append(number+i)

#check if a number is duplicated 
print(len(final_random_list) == len(set(final_random_list)))

True


In [14]:
#save the random list
np.save('../data/formatted/final_random_list.npy', final_random_list)

In [16]:
#split dataset and validation using the random list
dataset_test = dataset[final_random_list]
midi_test = midi_dataset[final_random_list]
meta_test = meta_dataset[final_random_list]

dataset_train = np.delete(dataset, final_random_list, axis=0)
midi_train = np.delete(midi_dataset, final_random_list, axis=0)

In [17]:
#save the train and test dataset
#test
np.save('../data/formatted/dataset_test.npy', dataset_test)
np.save('../data/formatted/midi_test.npy', midi_test)
np.save('../data/formatted/meta_test.npy', meta_test)
#train
np.save('../data/formatted/dataset_train.npy', dataset_train)
np.save('../data/formatted/midi_train.npy', midi_train)

In [23]:
import random
#create a file with shuffled reference index
def createWindowedShuffleReference(type, size, window, save = False):
    s = np.arange(0, size, 1)
    #num = np.arange(0, len(data)/10, 1)
    np.random.shuffle(s)

    n = int(size/window)
    numlist = random.sample(range(n), n)
    numlist = np.array(numlist)
    numlist = numlist * window

    m = np.max(numlist)
    l_ref = size-window
    print('real:', size, 'max:', m, 'length_ref:',l_ref)

    if m != l_ref:
        rest = m - l_ref
        numlist = numlist - rest

    ref = []
    for num in numlist:
        if num == 0:
            print("OK")
        for i in range(0,window):
            ref.append(num+i)

    #return the shuffled list
    if save:
        np.savetxt("../data/shuffle_" + type + ".txt", ref, fmt='%i', delimiter=" ", header='Array shape: ('+str(size)+', 1)')
    return ref

In [21]:
def getData(folder, name):
    data_path = folder + '/' + name
    data = np.loadtxt(data_path)
    f = open(data_path, "r")
    format = f.readline().replace('# Array shape: (', '').replace('\n', '').replace(')', '')
    format = np.array(format.split(', ')).astype(int)
    f.close()
    return data, format

In [24]:
train_dataset = np.load('../data/formatted/dataset_train.npy', allow_pickle=True)
test_dataset = np.load('../data/formatted/dataset_test.npy', allow_pickle=True)

train_midi = np.load('../data/formatted/midi_train.npy', allow_pickle=True)
test_midi = np.load('../data/formatted/midi_test.npy', allow_pickle=True)

print(train_dataset.shape, train_midi.shape, test_dataset.shape, test_midi.shape)

BATCH_SHUFFLE_SIZE = 1
ref = createWindowedShuffleReference("train", len(train_dataset), BATCH_SHUFFLE_SIZE, True)
ref_test = createWindowedShuffleReference("test", len(test_dataset), BATCH_SHUFFLE_SIZE, True)

# first shuffle the train dataset
shuffle_train, format_train = getData('../data', 'shuffle_train.txt')
shuffle_train = shuffle_train.reshape(format_train[0], ).astype(int)
shuffle_train = shuffle_train.tolist()
dataset = train_dataset[shuffle_train]
midiDataset = train_midi[shuffle_train]

#second shuffle the test dataset
shuffle_test, format_test = getData('../data', 'shuffle_test.txt')
shuffle_test = shuffle_test.reshape(format_test[0], ).astype(int)
shuffle_test = shuffle_test.tolist()
validation = test_dataset[shuffle_test]
midi_validation = test_midi[shuffle_test]

(50772, 1024, 2) (50772, 1024, 3) (5640, 1024, 2) (5640, 1024, 3)
real: 50772 max: 50771 length_ref: 50771
OK
real: 5640 max: 5639 length_ref: 5639
OK


In [26]:
np.save('../data/shuffled/dataset_train.npy', dataset)
np.save('../data/shuffled/midi_train.npy', midiDataset)
np.save('../data/shuffled/dataset_test.npy', validation)
np.save('../data/shuffled/midi_test.npy', midi_validation)

Checkpoint

In [1]:
%reload_ext autoreload
%autoreload 2
from utils import *

max_length = 1024
id = 0
tokens = np.load('../data/formatted/tokens.npy', allow_pickle=True)
train = np.load('../data/shuffled/dataset_train.npy', allow_pickle=True)
test = np.load('../data/shuffled/dataset_test.npy', allow_pickle=True)
midi_train = np.load('../data/shuffled/midi_train.npy', allow_pickle=True)
midi_test = np.load('../data/shuffled/midi_test.npy', allow_pickle=True)

In [2]:
#extract the first dimension of MIDI dataset, to get shape (n, 1024, 1)

midi_data_train = np.array([[chord[0] for chord in song] for song in midi_train])
midi_data_test = np.array([[chord[0] for chord in song] for song in midi_test])
  
print(train.shape, test.shape, midi_data_train.shape, midi_data_test.shape)

dataset = TokenDatasetMidi(train, midi_data_train,  max_length, tokens)
validation = TokenDatasetMidi(test, midi_data_test, max_length, tokens)

(50772, 1024, 2) (5640, 1024, 2) (50772, 1024, 8) (5640, 1024, 8)
data has 50772 pieces, 243 unique tokens.
data has 5640 pieces, 243 unique tokens.


In [31]:
import wandb
#wandb.login()
wandb.init(
    # set the wandb project where this run will be logged
    project="music_gpt_new_voicing",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 3e-5,
    "architecture": "Transformer - minGPT",
    "dataset": "chords from iRealPro",
    "epochs": 250,
    }
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [35]:
%reload_ext autoreload
%autoreload 2
from trainer import Trainer, TrainerConfig
from mingpt_utils import set_seed
from model import GPT, GPTConfig
import torch
print("Available devices: ", torch.cuda.device_count())

from torch.utils.tensorboard import SummaryWriter
from mingpt_utils import sample
torch.cuda.empty_cache()

Available devices:  2


In [56]:
epochs = 250
embedding = 192
heads = 4
layers = 4
batch_size = 64
learning_rate = 3e-5
num_workers = 4
midi_vocab = 128

mconf = GPTConfig(len(tokens), dataset.block_size, midi_vocab, n_layer=layers, n_head=heads, n_embd=embedding)
session_model = GPT(mconf)
MODEL_NAME = "../models/model_"+ "epochs->" + str(epochs) + "_heads->" + str(heads) + "_embd->" + str(embedding) + "_batch->" + str(batch_size) + "_new_midi_embeddings"
print(MODEL_NAME)

session_model = load_model(MODEL_NAME, session_model)

if (session_model == None):
    #mconf = GPTConfig(len(tokens), dataset.block_size, n_layer=layers, n_head=heads, n_embd=embbedings)
    session_model = GPT(mconf)
    tconf = TrainerConfig(max_epochs=epochs, 
                          batch_size=batch_size, 
                          learning_rate=learning_rate, 
                          num_workers=num_workers
                          )
    writer = SummaryWriter(log_dir='../runs/'+'logs') 
    trainer = Trainer(session_model, dataset, validation, tconf, writer)
    trainer.train()
    save_model(MODEL_NAME, session_model)
    # [optional] finish the wandb run, necessary in notebooks
    wandb.finish()